# Homework: Named entity recognition

Для заданной тестовой выборки построить модель для обнаружения и классификации именованных сущностей (named entities). На базе корпуса CoNLL 2002.  

Чем больше baseline'ов вы превзойдете, тем выше ваша оценка
Метрика качества f1 (f1_macro) (чем выше, тем лучше)
 
baseline 1: 0.0604      random labels  
baseline 2: 0.3966      PoS features + logistic regression  
baseline 3: 0.7559      word2vec cbow embedding + baseline 2 + svm    

Пока мы рассмотрели только линейные модели - поэтому в примерах есть только они. Желательно при решении домашнего задания пользоваться линейными моделями. Таким образом, основные цели задания - feature engineering, hyperparam tuning & model selection.

! Your results must be reproducible. Если ваша модель - стохастическая, то вы явно должны задавать все seed и random_state в параметрах моделей  
! Вы должны использовать df_test только для измерения качества конечной обученной модели. 

bonus, think about:  
1. how can you exploit that words belong to some sentence?
2. why we selected f1 score with macro averaging as our classification quality measure? What other metrics are suitable   

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import model_selection
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.svm import LinearSVC
import scipy.sparse as sp

from gensim.models.word2vec import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin
from collections import defaultdict
from nltk.tokenize import sent_tokenize

from sklearn.preprocessing import OneHotEncoder
from sklearn.dummy import DummyClassifier

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


SEED=1337

C:\Users\Андрей\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
df = pd.read_csv('ner_short.csv', index_col=0)
df.head()

,next-next-pos,next-next-word,next-pos,next-word,pos,prev-pos,prev-prev-pos,prev-prev-word,prev-word,sentence_idx,word,tag
0,NNS,demonstrators,IN,of,NNS,__START1__,__START2__,__START2__,__START1__,1.0,Thousands,O
1,VBP,have,NNS,demonstrators,IN,NNS,__START1__,__START1__,Thousands,1.0,of,O
2,VBN,marched,VBP,have,NNS,IN,NNS,Thousands,of,1.0,demonstrators,O
3,IN,through,VBN,marched,VBP,NNS,IN,of,demonstrators,1.0,have,O
4,NNP,London,IN,through,VBN,VBP,NNS,demonstrators,have,1.0,marched,O


In [3]:
# number of sentences
df.sentence_idx.max()

1500.0

In [4]:
# class distribution
df.tag.value_counts(normalize=True )

O        0.852828
B-geo    0.027604
B-gpe    0.020935
B-org    0.020247
I-per    0.017795
B-tim    0.016927
B-per    0.015312
I-org    0.013937
I-geo    0.005383
I-tim    0.004247
B-art    0.001376
I-gpe    0.000837
I-art    0.000748
B-eve    0.000628
I-eve    0.000508
B-nat    0.000449
I-nat    0.000239
Name: tag, dtype: float64

In [5]:
# sentence length
tdf = df.set_index('sentence_idx')
tdf['length'] = df.groupby('sentence_idx').tag.count()
df = tdf.reset_index(drop=False)

In [6]:
# encode categorial variables

le = LabelEncoder()
df['pos'] = le.fit_transform(df.pos)
df['next-pos'] = le.fit_transform(df['next-pos'])
df['next-next-pos'] = le.fit_transform(df['next-next-pos'])
df['prev-pos'] = le.fit_transform(df['prev-pos'])
df['prev-prev-pos'] = le.fit_transform(df['prev-prev-pos'])

In [7]:
df.head()

,sentence_idx,next-next-pos,next-next-word,next-pos,next-word,pos,prev-pos,prev-prev-pos,prev-prev-word,prev-word,word,tag,length
0,1.0,18,demonstrators,9,of,18,39,40,__START2__,__START1__,Thousands,O,48
1,1.0,33,have,18,demonstrators,9,18,39,__START1__,Thousands,of,O,48
2,1.0,32,marched,33,have,18,9,18,Thousands,of,demonstrators,O,48
3,1.0,9,through,32,marched,33,18,9,of,demonstrators,have,O,48
4,1.0,16,London,9,through,32,33,18,demonstrators,have,marched,O,48


In [8]:
# splitting
y = LabelEncoder().fit_transform(df.tag)

df_train, df_test, y_train, y_test = model_selection.train_test_split(df, y, stratify=y, 
                                                                      test_size=0.25, random_state=SEED, shuffle=True)
print('train', df_train.shape[0])
print('test', df_test.shape[0])

train 50155
test 16719


In [9]:
# some wrappers to work with word2vec

   
class Word2VecWrapper(TransformerMixin):
    def __init__(self, window=5,negative=5, size=100, iter=100, is_cbow=False, random_state=SEED):
        self.window_ = window
        self.negative_ = negative
        self.size_ = size
        self.iter_ = iter
        self.is_cbow_ = is_cbow
        self.w2v = None
        self.random_state = random_state
        
    def get_size(self):
        return self.size_

    def fit(self, X, y=None):
        """
        X: list of strings
        """
        sentences_list = [x.split() for x in X]
        self.w2v = Word2Vec(sentences_list, 
                            window=self.window_,
                            negative=self.negative_, 
                            size=self.size_, 
                            iter=self.iter_,
                            sg=not self.is_cbow_, seed=self.random_state)

        return self
    
    def has(self, word):
        return word in self.w2v

    def transform(self, X):
        """
        X: a list of words
        """
        if self.w2v is None:
            raise Exception('model not fitted')
        return np.array([self.w2v[w] if w in self.w2v else np.zeros(self.size_) for w in X ])

In [10]:
%%time
# we have to fit embedding models on whole dataset as they depend on word order

# notice, that our dataset has window=2
sentences_list = [x for x in sent_tokenize(' '.join(df.word))]
print(sentences_list[:5])

w2v_cbow = Word2VecWrapper(window=2, negative=5, size=300, iter=300, is_cbow=True, random_state=SEED)
w2v_cbow.fit(sentences_list)

['Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .', 'Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as " Bush Number One Terrorist " and " Stop the Bombings . "', 'They marched from the Houses of Parliament to a rally in Hyde Park .', 'Police put the number of marchers at 10,000 while organizers claimed it was 1,00,000 .', "The protest comes on the eve of the annual conference of Britain 's ruling Labor Party in the southern English seaside resort of Brighton ."]
Wall time: 15.3 s


In [11]:
embeding = w2v_cbow
encoder_pos = OneHotEncoder()
X_train = sp.hstack([
    embeding.transform(df_train.word),
    embeding.transform(df_train['next-word']),
    embeding.transform(df_train['next-next-word']),
    embeding.transform(df_train['prev-word']),
    embeding.transform(df_train['prev-prev-word']),
    encoder_pos.fit_transform(df_train[['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']])
    
])
X_test = sp.hstack([
    embeding.transform(df_test.word),
    embeding.transform(df_test['next-word']),
    embeding.transform(df_test['next-next-word']),
    embeding.transform(df_test['prev-word']),
    embeding.transform(df_test['prev-prev-word']),
    encoder_pos.transform(df_test[['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']])
])

In [13]:
# моя модель - word2vec cbow embedding + PoS features & LogisticRegression
# test 0.843024533396

params = {'solver': ['lbfgs'],
          'penalty': ['l2'],
          'tol': [0.0001, 0.01],
          'C': [1,1000],
          'class_weight': ['balanced',None],
          'max_iter': [500]}

model = model_selection.GridSearchCV(LogisticRegression(multi_class='multinomial',random_state=SEED),
                                     params,cv=3, scoring='f1_macro', verbose=2)
model.fit(X_train, y_train)

print('train', metrics.f1_score(y_train, model.predict(X_train), average='macro'))
print('test', metrics.f1_score(y_test, model.predict(X_test), average='macro'))

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] solver=lbfgs, C=1, tol=0.0001, penalty=l2, class_weight=balanced, max_iter=500 
[CV]  solver=lbfgs, C=1, tol=0.0001, penalty=l2, class_weight=balanced, max_iter=500, total= 9.6min
[CV] solver=lbfgs, C=1, tol=0.0001, penalty=l2, class_weight=balanced, max_iter=500 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  9.6min remaining:    0.0s


[CV]  solver=lbfgs, C=1, tol=0.0001, penalty=l2, class_weight=balanced, max_iter=500, total= 9.8min
[CV] solver=lbfgs, C=1, tol=0.0001, penalty=l2, class_weight=balanced, max_iter=500 
[CV]  solver=lbfgs, C=1, tol=0.0001, penalty=l2, class_weight=balanced, max_iter=500, total= 9.7min
[CV] solver=lbfgs, C=1, tol=0.01, penalty=l2, class_weight=balanced, max_iter=500 
[CV]  solver=lbfgs, C=1, tol=0.01, penalty=l2, class_weight=balanced, max_iter=500, total= 9.9min
[CV] solver=lbfgs, C=1, tol=0.01, penalty=l2, class_weight=balanced, max_iter=500 
[CV]  solver=lbfgs, C=1, tol=0.01, penalty=l2, class_weight=balanced, max_iter=500, total= 9.9min
[CV] solver=lbfgs, C=1, tol=0.01, penalty=l2, class_weight=balanced, max_iter=500 
[CV]  solver=lbfgs, C=1, tol=0.01, penalty=l2, class_weight=balanced, max_iter=500, total= 9.5min
[CV] solver=lbfgs, C=1, tol=0.0001, penalty=l2, class_weight=None, max_iter=500 
[CV]  solver=lbfgs, C=1, tol=0.0001, penalty=l2, class_weight=None, max_iter=500, total= 9.

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 228.3min finished


train 0.982491458079
test 0.843024533396


In [15]:
model.best_params_

{'C': 1,
 'class_weight': None,
 'max_iter': 500,
 'penalty': 'l2',
 'solver': 'lbfgs',
 'tol': 0.0001}